# Chamfer Approximation

In this notebook we attempt to approximate the chamfer similarity directly. First, we build an LSH forest for each document and query to find the closest matching vector before taking the dot product. Second, we use an LSH forest that encorporates multiple documents together and perform this same estimation.

In [24]:
import numpy as np
import pytest


def chamfer(queries: np.ndarray, vectors: np.ndarray):
        """Takes two sets of vectors and calculates chamfer"""

        # (n, m) matrix of all the pairwise dot products
        dot_products = queries @ vectors.T

        # sum the max value for each query (row)
        chamfer = np.sum(np.max(dot_products, axis=1))
        return chamfer

# Quick test
A = np.array([[1, 0]], dtype=float)
B = np.array([[1, 0], [np.sqrt(.5), np.sqrt(.5)]], dtype=float)
C = np.array([[1, 0], [0, 1], [np.sqrt(.1), np.sqrt(.9)], [np.sqrt(.5), np.sqrt(.5)]], dtype=float)

assert chamfer(A, B) == pytest.approx(1.0)
assert chamfer(B, A) == pytest.approx(1 + np.sqrt(.5))
assert chamfer(B, C) == pytest.approx(2)
assert chamfer(B, B) == pytest.approx(2)
assert chamfer(C, C) == pytest.approx(4)

In [ ]:
from shared.lsh_forest import LSHForest, MultiDocLSHForest, BitSamplingLSH

n = 10                                  # number of documents
q = 16                                  # vectors per query
m = 32                                  # vectors per document
d = 64                                  # dimension per vector
l = 10                                  # trees per forest
k = 10                                  # pivots to keep per node in tree
km = 64                                 # max depth of each tree
a = 15                                  # number of neighbors to retrieve per query
rng = np.random.default_rng(42)


# Get normalized document and query vectors
vectors = rng.normal(size=(n, m, d)).astype(np.float32)
vectors /= np.linalg.norm(vectors, axis=-1, keepdims=True)

queries = rng.normal(size=(q, d)).astype(np.float32)
queries /= np.linalg.norm(queries, axis=-1, keepdims=True)

distance = lambda a, b: -np.dot(a, b)


# Build the forests
single_doc_forests = [LSHForest(BitSamplingLSH(d), l, k, km) for _ in range(n)]
for doc, forest in enumerate(single_doc_forests):
        forest.batch_insert(vectors[doc])

multi_doc_forest = MultiDocLSHForest(BitSamplingLSH(d), l, k, km)
multi_doc_forest.batch_insert(vectors)


# Baseline
best = (0, 0)
for document, doc_vecs in enumerate(vectors):
        sim = chamfer(queries, doc_vecs)
        if sim >= best[0]:
                best = (float(sim), document)
print(best)


# Evaluate single-doc
matches = np.empty((n, q, d), dtype=np.float32)
for document, forest in enumerate(single_doc_forests):
        for i, query in enumerate(queries):
                idx = forest.query(query, a, dist=distance)[0][0]
                matches[document, i] = forest.data[idx]
sims = np.tensordot(matches, queries, axes=([1, 2], [0, 1]))
best_doc = int(np.argmax(sims))
best = (float(sims[best_doc]), best_doc)
print(best)


# Evaluate multi-doc
matches.fill(0)
for i, query in enumerate(queries):
        results = multi_doc_forest.query(query, a, dist=distance)
        for document, result in enumerate(results):
                idx, _ = result[0]
                matches[document, i] = multi_doc_forest.data[document][idx]
sims = np.tensordot(matches, queries, axes=([1, 2], [0, 1]))
best_doc = int(np.argmax(sims))
best = (float(sims[best_doc]), best_doc)
print(best)



(4.590232849121094, 0)
(3.535553455352783, 7)


TypeError: cannot unpack non-iterable int object

In [ ]:
# TODO: Test with many documents and varying hyperparameters
# Some graphs of convergence would be nice